In [15]:
import pandas as pd

In [16]:
data = pd.read_csv('../data/snodgrass.csv')

In [17]:
data

,author,total
0,twain,0.225
1,twain,0.262
2,twain,0.217
3,twain,0.240
4,twain,0.230
5,twain,0.229
6,twain,0.235
7,twain,0.217
8,snodgrass,0.209
9,snodgrass,0.205


In [18]:
data.groupby('author').aggregate(['mean', 'count'])

total      
               mean count
author                   
snodgrass  0.209700    10
twain      0.231875     8

$$t_{obs}$$ is 

$$|.2097 - .231875| = .022175$$

In [24]:
t_obs = .022175

## Permutation test

The full set of observations in $$X_1,\ldots,X_{10}, Y_1,\ldots,Y_{8}$$

For all 18! permutations of the data, compute

$$|(\bar{X}) - (\bar{Y})|$$

where $$(\bar{X})$$ is the mean of the first 10 observations, and $$(\bar{Y})$$ is the mean of the last 8.

In [19]:
from itertools import permutations

In [22]:
obs = list(data['total'])

In [42]:
import numpy as np

In [48]:
perms = (p for p in permutations([1,2,3]))
for p in perms:
    print(np.mean(p[:2]) - np.mean(p[2:]))

-1.5
0.0
-1.5
1.5
0.0
1.5


In [103]:
# there are too many permutations to count all of them, so we set some limit N
# on the total number of observed permutations


def permutation_test(obs, m, c, N):
    i = 0 # counter for T>c
    
    # adjust m so there's at least one observation in each sample
    m = min(m, len(obs)-1)

    # evaluate permutations
    for j in range(N):
        p = np.random.permutation(obs)
        # calculate stat
        T = np.mean(p[:m]) - np.mean(p[m:])
        # count if > c
        if T > c:
            i += 1
            
    # i/N approximates the p-value as N -> N!
    return i/N


In [104]:
permutation_test(obs, 10, t_obs, N=100000)

0.00023

## Wald test

The Wald test of two means can be constructed as (see example 10.7)

$$ W = \frac{\bar{X}-\bar{Y}}{\sqrt{\frac{s_x^2}{m}+\frac{s_y^2}{n}}} $$

In [107]:
X = data[data['author']=='snodgrass']['total']
Y = data[data['author']=='twain']['total']

x_bar = np.mean(X)
m = len(X)
s_x = np.std(X)

y_bar = np.mean(Y)
n = len(Y)
s_y = np.std(Y)


In [116]:
W = (x_bar - y_bar)/np.sqrt(s_x**2/m + s_y**2/n)

Finally we just need to test that $$W > z_{\alpha/2}$$


In [119]:
np.abs(W) > 1.96

True

We can get the p-value:

In [120]:
from scipy.stats import norm

In [123]:
norm.cdf(W)

3.9963324780729e-05

To get the CI we need $$\hat{\theta}$$, which here is $$\bar{X}-\bar{Y}$$, and se_hat, which is $$\sqrt{\frac{s_x^2}{m}+\frac{s_y^2}{n}}$$.

In [124]:
theta_hat = x_bar-y_bar
se_hat = np.sqrt(s_x**2/m + s_y**2/n)

In [125]:
(theta_hat, theta_hat+2*se_hat, theta_hat-2*se_hat)

(-0.022175, -0.010931838189370394, -0.033418161810629607)

## Conclusions

Our p-value from both tests is very small, which is strong evidence for rejecting the null that the means of the two samples are the same.

Additionally, the confidence intervals do not overlap, which again is evidence against $$H_0$$